# External Memory Management Exercise

Given the following Python code that implements device memory allocation, freeing, and stats (free / total memory), implement an EMM plugin that enables Numba to manage memory using them:

```python
from ctypes import CDLL, POINTER, byref, c_void_p, c_size_t


# Open the CUDA runtime DLL and create bindings for the cudaMalloc, cudaFree,
# and cudaMemGetInfo functions.

cudart = CDLL('libcudart.so')

cudaMalloc = cudart.cudaMalloc
cudaMalloc.argtypes = [POINTER(c_size_t), c_size_t]

cudaFree = cudart.cudaFree
cudaFree.argtypes = [c_void_p]

cudaMemGetInfo = cudart.cudaMemGetInfo
cudaMemGetInfo.argtypes = [POINTER(c_size_t), POINTER(c_size_t)]


# Python functions for allocation, deallocation, and memory info

def my_alloc(size):
    """
    Allocate `size` bytes of device memory and return a device pointer to the
    allocated memory.
    """
    ptr = c_size_t()
    ret = cudaMalloc(byref(ptr), size)
    if ret:
        raise RuntimeError(f'Unexpected return code {ret} from cudaMalloc')
    return ptr


def my_free(ptr):
    """
    Free device memory pointed to by `ptr`.
    """
    cudaFree(ptr)


def my_memory_info():
    free = c_size_t()
    total = c_size_t()
    cudaMemGetInfo(byref(free), byref(total))
    return free, total
```


## Hints

### Plugin class

You will need to create a class for your EMM Plugin.

- Since the above functions only manage device memory, you will probably want to base it on `HostOnlyCUDAMemoryManager`.
- You may also want to use the `GetIpcHandleMixin`, so that you don't have to implement `get_ipc_handle()` yourself.


### Testing by setting the allocator

You can do a simple test of your allocator by setting the memory manager to your plugin class with `numba.cuda.set_memory_manager(<plugin class>)`. If you insert logging `print()` calls into you `memalloc()`, `initialize()`, etc., methods, you should see that those functions are called when Numba needs to allocate memory, for example using `cuda.device_array()` or `cuda.to_device()`. This will help you start to test and debug your implementation.


### Testing using the Numba testsuite

You can run the Numba CUDA testsuite with:

```
python -m numba.runtests numba.cuda.tests
```

This exercises memory functionality quite extensively, but uses Numba's internal memory management by default. To run the testsuite with your plugin, you need to add a global to the module in which your EMM Plugin is defined:

```
_numba_memory_manager = <EMM Plugin class>
```

Then, you can run the testsuite with

```
NUMBA_CUDA_MEMORY_MANAGER=<plugin module> python -m numba.runtests numba.cuda.tests
```

and Numba will automatically use your EMM Plugin instead of its internal memory management. If all tests with the internal memory management pass on your system, then it should also be possible to get a 100% pass rate using an EMM plugin based on the code above.

## Solution

An example solution to this exercise is in `session-5/examples/simple_emm_plugin.py`. It can be used both standalone (`python simple_emm_plugin.py`) for a simple test, or with the `NUMBA_CUDA_MEMORY_MANAGER` environment variable for running the Numba testsuite.